In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import csv

# 2. Train and create a YOLO model by randomly taking 400 images from train data which can detect graffiti in the image

In [ ]:
%cd /content/drive/MyDrive/phong/yolov5

/content/drive/MyDrive/phong/yolov5


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
!python train.py --data data/mydataset.yaml --epochs 2 --batch-size 16 --weights yolov5m.pt

2024-09-21 15:11:57.853762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 15:11:57.873973: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 15:11:57.880363: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=, data=data/mydataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=2, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=Fal

# 3. Randomly take 40 images from test data and compute IoU for each and generate a CSV  file containing 3 columns [image_name, confidence value, IoU value]. If no graffiti is  detected for an image then its IoU will be 0.

In [ ]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/exp/weights/best.pt --source /content/drive/MyDrive/phong/data/images/test_40 --save-txt --save-conf

detect: weights=['/content/drive/MyDrive/phong/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/phong/data/images/test_40, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-368-gb163ff8d Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/40 /content/drive/MyDrive/phong/data/images/test_40/IMG_0233.JPG: 640x480 1 graffiti, 52.8ms
image 2/40 /content/drive/MyDrive/phong/data/images/test_40/IMG_1251.jpg: 640x480 (no detections), 20.2ms
i

In [35]:
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xi1 = max(x1 - w1 / 2, x2 - w2 / 2)
    yi1 = max(y1 - h1 / 2, y2 - h2 / 2)
    xi2 = min(x1 + w1 / 2, x2 + w2 / 2)
    yi2 = min(y1 + h1 / 2, y2 + h2 / 2)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area

    return inter_area / (union_area + 1e-16)

In [36]:
# save average detection for confidence and IOU

def process_labels(ground_truth_dir, predict_dir, output_csv):
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["image_name", "confidence_value", "IoU_value"])

        # Collect all ground truth files
        gt_files = [f for f in os.listdir(ground_truth_dir) if f.endswith('.txt')]

        for gt_file in gt_files:
            image_name = gt_file.replace('.txt', '')

            # Read ground truth boxes once
            gt_path = os.path.join(ground_truth_dir, gt_file)
            with open(gt_path, 'r') as f:
                ground_truth_data = [list(map(float, line.strip().split()[1:5])) for line in f.readlines()]

            # Check for the corresponding prediction file
            pred_path = os.path.join(predict_dir, gt_file)
            if os.path.exists(pred_path):
                with open(pred_path, 'r') as f:
                    predict_data = [line.strip().split() for line in f.readlines()]

                if not predict_data:
                    # No predictions, write IoU and confidence as 0
                    writer.writerow([image_name, 0, 0])
                    continue

                iou_sum = 0
                confidence_sum = 0
                num_detections = 0

                # Process each predicted box
                for pred_values in predict_data:
                    confidence = float(pred_values[-1])
                    pred_box = list(map(float, pred_values[1:5]))  # Predicted box
                    # Calculate IoU against all ground truth boxes and keep the maximum IoU
                    max_iou = max(calculate_iou(pred_box, gt_box) for gt_box in ground_truth_data)

                    # Accumulate IoU and confidence
                    iou_sum += max_iou
                    confidence_sum += confidence
                    num_detections += 1

                # Calculate the average IoU and confidence for the image
                average_iou = iou_sum / num_detections if num_detections > 0 else 0
                average_confidence = confidence_sum / num_detections if num_detections > 0 else 0

                # Write the average values to the CSV
                writer.writerow([image_name, average_confidence, average_iou])

            else:
                # If no prediction file, write IoU and confidence as 0
                writer.writerow([image_name, 0, 0])

In [ ]:
ground_truth_dir = '/content/drive/MyDrive/phong/data/labels/test_40'
predict_dir = '/content/drive/MyDrive/phong/yolov5/runs/detect/exp/labels'
output_csv = '/content/drive/MyDrive/phong/iou_results.csv'
process_labels(ground_truth_dir, predict_dir, output_csv)

In [ ]:
def check_IOU(csv_file, threshold=0.9):
    df = pd.read_csv(csv_file)
    count_above_threshold = (df['IoU_value'] > threshold).sum()
    percentage = count_above_threshold / len(df)

    # Check if 80% or more of the test images meet the IoU condition
    if percentage >= 0.8:
        return "The condition is met: {:.2%} of the test set has IoU > 90%.".format(percentage)
    else:
        return "The condition is not met: {:.2%} of the test set has IoU > 90%. Continue training.".format(percentage)

In [ ]:
check_IOU('/content/drive/MyDrive/phong/iou_results.csv')

'The condition is not met: 22.50% of the test set has IoU > 90%. Continue training.'

**FIRST ITERATION**

In [ ]:
!python train.py --data data/mydataset.yaml --epochs 20 --batch-size 16 --weights yolov5m.pt --name iteration_1

2024-09-21 15:13:35.006193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 15:13:35.027108: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 15:13:35.034827: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=, data=data/mydataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=Fa

In [ ]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_1/weights/best.pt --source /content/drive/MyDrive/phong/data/images/test_40 --save-txt --save-conf --name iteration_1

detect: weights=['/content/drive/MyDrive/phong/yolov5/runs/train/iteration_1/weights/best.pt'], source=/content/drive/MyDrive/phong/data/images/test_40, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=iteration_1, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-368-gb163ff8d Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/40 /content/drive/MyDrive/phong/data/images/test_40/IMG_0233.JPG: 640x480 1 graffiti, 51.9ms
image 2/40 /content/drive/MyDrive/phong/data/images/test_40/IMG_1251.jpg: 640x480 1 graffit

In [37]:
ground_truth_dir = '/content/drive/MyDrive/phong/data/labels/test_40'
predict_dir = '/content/drive/MyDrive/phong/yolov5/runs/detect/iteration_1/labels'
output_csv = '/content/drive/MyDrive/phong/iou_results_iteration1.csv'
process_labels(ground_truth_dir, predict_dir, output_csv)
check_IOU('/content/drive/MyDrive/phong/iou_results_iteration1.csv')

'The condition is not met: 72.50% of the test set has IoU > 90%. Continue training.'

**SECOND ITERATION**

In [39]:
!python train.py --data data/mydataset.yaml --epochs 20 --batch-size 16 --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_1/weights/best.pt --name iteration_2

2024-09-21 15:28:42.588147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 15:28:42.621296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 15:28:42.631061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=/content/drive/MyDrive/phong/yolov5/runs/train/iteration_1/weights/best.pt, cfg=, data=data/mydataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None

In [40]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_2/weights/best.pt --source /content/drive/MyDrive/phong/data/images/test_40 --save-txt --save-conf --name iteration_2

detect: weights=['/content/drive/MyDrive/phong/yolov5/runs/train/iteration_2/weights/best.pt'], source=/content/drive/MyDrive/phong/data/images/test_40, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=iteration_2, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-368-gb163ff8d Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/40 /content/drive/MyDrive/phong/data/images/test_40/IMG_0233.JPG: 640x480 1 graffiti, 53.7ms
image 2/40 /content/drive/MyDrive/phong/data/images/test_40/IMG_1251.jpg: 640x480 1 graffiti, 20.2ms
image 3/40 /content/drive/MyDriv

In [41]:
ground_truth_dir = '/content/drive/MyDrive/phong/data/labels/test_40'
predict_dir = '/content/drive/MyDrive/phong/yolov5/runs/detect/iteration_2/labels'
output_csv = '/content/drive/MyDrive/phong/iou_results_iteration2.csv'
process_labels(ground_truth_dir, predict_dir, output_csv)
check_IOU('/content/drive/MyDrive/phong/iou_results_iteration2.csv')

'The condition is met: 80.00% of the test set has IoU > 90%.'

**THIRD ITERATION**

In [ ]:
!python train.py --data data/mydataset.yaml --epochs 20 --batch-size 32 --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_2/weights/best.pt --name iteration_3

In [ ]:
!python detect.py --weights D:/phong/yolov5/runs/train/iteration_3/weights/best.pt --source D:/phong/newdata/images/test_40 --save-txt --save-conf --name iteration_3

In [ ]:
ground_truth_dir = '/content/drive/MyDrive/phong/newdata/labels/test_40'
predict_dir = '/content/drive/MyDrive/phong/yolov5/runs/detect/iteration_3/labels'
output_csv = '/content/drive/MyDrive/phong/iou_results_iteration3.csv'
process_labels(ground_truth_dir, predict_dir, output_csv)
check_IOU('/content/drive/MyDrive/phong/iou_results_iteration.csv')

**FOURTH ITERATION**

In [ ]:
!python train.py --data data/mydataset.yaml --epochs 20 --batch-size 32 --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_3/weights/best.pt --name iteration_4

In [ ]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_4/weights/best.pt --source /content/drive/MyDrive/phong/newdata/images/test_40 --save-txt --save-conf --name iteration_4

In [ ]:
ground_truth_dir = 'D:/phong/newdata/labels/test_40'
predict_dir = 'D:/phong/yolov5/runs/detect/iteration_3/labels'
output_csv = 'D:/phong/iou_results_iteration3.csv'
process_labels(ground_truth_dir, predict_dir, output_csv)
check_IOU('D:/phong/iou_results_iteration3.csv')

# DETECT ON VIDEOS

In [ ]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_4/weights/best.pt --source /content/drive/MyDrive/phong/videos/854181-hd_1920_1080_25fps.mp4

In [ ]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_4/weights/best.pt --source /content/drive/MyDrive/phong/videos/3413463-hd_1920_1080_30fps.mp4

In [ ]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_4/weights/best.pt --source /content/drive/MyDrive/phong/videos/4543511-hd_1080_1920_25fps.mp4

In [ ]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_4/weights/best.pt --source /content/drive/MyDrive/phong/videos/6537567-hd_1920_1080_24fps.mp4

In [ ]:
!python detect.py --weights /content/drive/MyDrive/phong/yolov5/runs/train/iteration_4/weights/best.pt --source /content/drive/MyDrive/phong/videos/9724130-hd_1440_1080_30fps.mp4